In [1]:
#
#    Notebook de cours MAP412 - Chapitre 9 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Onde progressive - Nagumo
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Onde progressive - Nagumo

In [ ]:
from dataclasses import dataclass
import time
import numpy as np
from scipy.sparse import diags, eye
from scipy.integrate import solve_ivp
from scipy.sparse.linalg import spsolve
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

## Equation de Nagumo

On considère l'équation de Nagumo :

$$
\partial_t u - D \, \partial_{xx} u = k \, u^2 (1 - u) \;\; \text{pour} \; -L < x < L\\     
$$

avec $L=50$, $k=1$ et $d=1$. Il est aisé de tester les méthodes avec une onde plus raide mais avec la même vitesse de propagation ($k=10$ et $d=0.1$).

On considère 2001 point de maillage menant à $\Delta x = 1/20$. La vitesse de l'onde est $c=1/\sqrt{2}$.

In [ ]:
def show_sol_and_err(tini, tsol, x, yini, ysol, yref):

    err = np.abs(yref-ysol)

    fig = make_subplots(rows=2, cols=1, subplot_titles=("Solution", "Erreur"), vertical_spacing=0.15)
    
    marker = dict(symbol='x', color='grey')
    fig.add_trace(go.Scattergl(x=x, y=yini, name=f'sol at t={tini}', mode='markers', marker=marker), row=1, col=1)
    marker = dict(symbol='x', color='rgb(76,114,176)')
    fig.add_trace(go.Scattergl(x=x, y=yref, name=f'sol at t={tsol}', mode='markers', marker=marker), row=1, col=1)
    
    fig.add_trace(go.Scattergl(x=x, y=err, showlegend=False, mode='markers', marker=marker), row=2, col=1)
    
    fig.update_layout(height=800)
    fig.update_yaxes(exponentformat='e')
    fig.show()

In [ ]:
class nagumo_model:

    def __init__(self, k, d, xmin, xmax, nx) :
        self.k = k
        self.d = d
        self.xmin = xmin
        self.xmax = xmax
        self.nx = nx
        self.dx = (xmax-xmin)/(nx-1)

        # construction de la matrice creuse de diffusion
        doverdxdx = d/(self.dx**2)
        diag = np.repeat(-2*doverdxdx,nx)
        diag_x = np.repeat([2*doverdxdx, doverdxdx], [1, nx-2])
        self.a = diags([np.flip(diag_x), diag, diag_x], [-1, 0, 1])

    def fcn(self, t, y):
        k = self.k
        return self.a.dot(y) + k*y*y*(1-y)

    def fcn_diff(self, t, y):        
        return self.a.dot(y)
    
    def fcn_reac(self, t, y):
        k = self.k  
        return k*y*y*(1 - y)
    
    def fcn_exact(self, t):
        k = self.k
        d = self.d
        xmin = self.xmin
        xmax = self.xmax
        nx = self.nx
        dx = self.dx
        x0 = -10.

        v = (1./np.sqrt(2.))*(np.sqrt(k*d))
        cst  = -(1./np.sqrt(2.))*(np.sqrt(k/d))

        x = np.linspace(xmin, xmax, nx)
        y = np.exp(cst*(x-x0-v*t)) / (1. + np.exp(cst*(x-x0-v*t)))
        return y

## Paramètres du problème

In [ ]:
k = 1.
d = 1.

xmin = -50.
xmax = 50.
nx = 2001
tini = 0.0
tend = 10.0

nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
dx = nm.dx
x = np.linspace(xmin, xmax, nx)
fcn_exact = nm.fcn_exact
fcn = nm.fcn
fcn_reac = nm.fcn_reac
fcn_diff = nm.fcn_diff
a = nm.a

yini = fcn_exact(tini)
yexa = fcn_exact(tend)

## Solution quasi exacte

In [ ]:
tol = 1.e-9

t1 = time.time()
sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=[tend], rtol=tol, atol=1.e-3*tol)
t2 = time.time()
print(f"Time to solve : {t2-t1} s")

yref = sol_ref.y[:,0]

err = np.abs(yref-yexa)
print(f"Norme de l'erreur par rapport à la solution exacte : {np.linalg.norm(err)/np.sqrt(nx-1):.10e}")

show_sol_and_err(tini, tend, x, yini, yref, yexa)

## Méthode de Runge et Kutta d'ordre 2 (Heun)

In [ ]:
@dataclass
class ode_result:
    t: np.ndarray 
    y: np.ndarray
    
def heun(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = yini

    for it, tn  in enumerate(t[:-1]):
        k1 = fcn(tn, yn)
        k2 = fcn(tn + dt, yn + dt*k1)
        yn = yn + dt/2*(k1+k2)

    return ode_result(t, yn)

In [ ]:
nt = 10001
dt = (tend-tini)/(nt-1)
print(f"dt = {dt}")

t1 = time.time()
sol = heun(tini, tend, nt, yini, fcn)
t2 = time.time()
print(f"Time to solve : {t2-t1} s")

err = np.abs(yref-sol.y)
print(f"Norme de l'erreur par rapport à la solution quasi-exacte : {np.linalg.norm(err)/np.sqrt(nx-1):.10e}")

show_sol_and_err(tini, tend, x, yini, sol.y, yref)

## Méthode IMEX basée sur des méthodes de Runge-Kutta additives

### IMEX - RK d'ordre 2 à 2 étages 

In [ ]:
def imex_rk22(tini, tend, nt, yini, a, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = yini

    lamb = 1 - np.sqrt(2)/2
    #mat = sparse.eye(yini.size) - lamb*dt*a
    mat = eye(yini.size) - lamb*dt*a

    for it, tn  in enumerate(t[:-1]):
        
        y1 = spsolve(mat.tocsr(), yn)

        rhs = yn + dt*fcn(tn,y1) + dt*(1-2*lamb)*a.dot(y1)
        y2 = spsolve(mat.tocsr(), rhs)

        yn = yn + 0.5*dt*(fcn(tn,y1) + fcn(tn,y2)) + 0.5*dt*a.dot(y1+y2)

    return ode_result(t, yn)

In [ ]:
nt = 1001
dt = (tend-tini)/(nt-1)
print(f"dt = {dt}")

t1 = time.time()
sol = imex_rk22(tini, tend, nt, yini, a, fcn_reac)
t2 = time.time()
print(f"Time to solve : {t2-t1} s")

err = np.abs(yref-sol.y)
print(f"Norme de l'erreur par rapport à la solution quasi-exacte : {np.linalg.norm(err)/np.sqrt(nx-1):.10e}")

show_sol_and_err(tini, tend, x, yini, sol.y, yref)

### IMEX - RK d'ordre 2 à 3 étages 

In [ ]:
def imex_rk23(tini, tend, nt, yini, a, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = yini

    lamb = 1 - np.sqrt(2)/2
    mat = eye(yini.size) - lamb*dt*a

    for it, tn  in enumerate(t[:-1]):
        
        rhs = yn + lamb*dt*fcn(tn,yn)
        y2 = spsolve(mat.tocsr(), rhs)
        
        rhs = yn + (lamb-1)*dt*fcn(tn,yn) + 2*(1-lamb)*dt*fcn(tn,y2) + dt*(1-2*lamb)*a.dot(y2)
        y3 = spsolve(mat.tocsr(), rhs)

        yn = yn + 0.5*dt*(fcn(tn,y2) + fcn(tn,y3)) + 0.5*dt*a.dot(y2+y3)

    return ode_result(t, yn)

In [ ]:
nt = 1001
dt = (tend-tini)/(nt-1)
print(f"dt = {dt}")

t1 = time.time()
sol = imex_rk23(tini, tend, nt, yini, a, fcn_reac)
t2 = time.time()
print(f"Time to solve : {t2-t1} s")

err = np.abs(yref-sol.y)
print(f"Norme de l'erreur par rapport à la solution quasi-exacte : {np.linalg.norm(err)/np.sqrt(nx-1):.10e}")

show_sol_and_err(tini, tend, x, yini, sol.y, yref)

## Méthode de splitting (Strang)

In [ ]:
def strang(tini, tend, nt, yini, fcn_a, fcn_b):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)
    
    yn = yini
    
    tol = 1.e-8
    
    for it, tn  in enumerate(t[:-1]):
        sol = solve_ivp(fcn_b, (tn, tn+(dt/2)), yn, method="RK45", rtol=tol, atol=tol)
        sol = solve_ivp(fcn_a, (tn, tn+dt), sol.y[:,-1], method="RK45", rtol=tol, atol=tol)
        sol = solve_ivp(fcn_b, (tn+(dt/2), tn+dt), sol.y[:,-1], method="RK45", rtol=tol, atol=tol)
        yn = sol.y[:,-1]

    return ode_result(t, yn)

In [ ]:
nt = 1001
dt = (tend-tini)/(nt-1)
print(f"dt = {dt}")

t1 = time.time()
sol = strang(tini, tend, nt, yini, fcn_diff, fcn_reac)
t2 = time.time()
print(f"Time to solve : {t2-t1} s")

err = np.abs(yref-sol.y)
print(f"Norme de l'erreur par rapport à la solution quasi-exacte : {np.linalg.norm(err)/np.sqrt(nx-1):.10e}")

show_sol_and_err(tini, tend, x, yini, sol.y, yref)